In [1]:
import pandas as pd, numpy as np, sklearn as sklm
import tensorflow as tf, sqlalchemy 
from sklearn.linear_model import LogisticRegression, LinearRegression 
import joblib 
pd.set_option('display.max_columns', None)
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('functions.py'), '..', '..', 'PY')))
import functions 
np.suppress

In [18]:
db_path = 'sqlite:///C:\\Users\\bchan\\OneDrive\\Personal Projects\\BID_Django\\ballindata\\DB\\ballbase.db' 

engine = sqlalchemy.create_engine(db_path) 
seasons = functions.generate_seasons(1979, 2024) 
master = pd.read_sql('master_as', con=engine)
ssn = pd.DataFrame() 
numeric_df = pd.read_sql('numeric_as', con=engine) 

,Player,Season,Pos,Age,Tm,GP,MPG,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORPG,DRPG,RPG,APG,SPG,BPG,ToPG,PF,PPG,T2P,T2PA,T3P,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB,DRtg,ORtg,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,AS


In [29]:
master[(master['Player']=='LeBron James') & (master['Season']=='2021-22')]

,Player,Season,Pos,Age,Tm,GP,MPG,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORPG,DRPG,RPG,APG,SPG,BPG,ToPG,PF,PPG,T2P,T2PA,T3P,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB,DRtg,ORtg,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,AS
18033,LeBron James,2021-22,C,37.0,LAL,56.0,37.2,11.4,21.8,52.4,2.9,8.0,35.9,8.6,13.8,62.0,59.0,4.5,6.0,75.6,1.1,7.1,8.2,6.2,1.3,1.1,3.5,2.2,30.3,479.0,773.0,161.0,448.0,349.0,59.0,396.0,640.0,1221.0,254.0,336.0,2084.0,63.0,121.0,1695.0,73.0,196.0,459.0,111.0,117.0,0.367,30.6,2.5,7.7,0.8,20.4,2.3,0.275,6.9,3.3,5.2,26.2,1.7,12.5,11.8,61.9,32.3,5.1,7.5,0.172,1.0


In [71]:
import joblib 
from keras.models import load_model 
models = {
    'logistic_regression': joblib.load("C:\\Users\\bchan\OneDrive\Personal Projects\BID_Django\\ballindata\IPYNB\Analyses\lr_model.pkl"), 
    'random_forest': joblib.load("C:\\Users\\bchan\OneDrive\Personal Projects\BID_Django\\ballindata\IPYNB\Analyses\\rf_model.pkl") 
} 
lr = joblib.load("C:\\Users\\bchan\OneDrive\Personal Projects\BID_Django\\ballindata\IPYNB\Analyses\lr_model.pkl") 
rf = joblib.load("C:\\Users\\bchan\OneDrive\Personal Projects\BID_Django\\ballindata\IPYNB\Analyses\\rf_model.pkl") 
nn = load_model("C:\\Users\\bchan\OneDrive\Personal Projects\BID_Django\\ballindata\IPYNB\Analyses\seq_model.keras") 

example = [numeric_df.iloc[18033].drop('AS').tolist()] 
pred_lr = lr.predict(example)[0] 
pred_rf = rf.predict(example)[0] 
pred_nn= nn.predict(tf.convert_to_tensor(example))[0][0].astype('float64')

print(f"{pred_lr} {pred_rf} {pred_nn}")
pred_lr == pred_rf 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1.0 1.0 0.936040997505188


True

# Upload to Database 

In [10]:
stat_names = numeric_df.columns.drop('AS').tolist()
stat_names 

['Age',
 'GP',
 'MPG',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORPG',
 'DRPG',
 'RPG',
 'APG',
 'SPG',
 'BPG',
 'ToPG',
 'PF',
 'PPG',
 'T2P',
 'T2PA',
 'T3P',
 'T3PA',
 'TAST',
 'TBLK',
 'TDRB',
 'TFG',
 'TFGA',
 'TFT',
 'TFTA',
 'TMP',
 'TORB',
 'TPF',
 'TPTS',
 'TSTL',
 'TTOV',
 'TTRB',
 'DRtg',
 'ORtg',
 '3PAr',
 'AST%',
 'BLK%',
 'BPM',
 'DBPM',
 'DRB%',
 'DWS',
 'FTr',
 'OBPM',
 'ORB%',
 'OWS',
 'PER',
 'STL%',
 'TOV%',
 'TRB%',
 'TS%',
 'USG%',
 'VORP',
 'WS',
 'WS/48']

In [17]:
output_html = '<div class="input-row">\n' 
for i in range(len(stat_names)):
    output_html += f'''<label for="{stat_names[i]}" class="input-label"> {stat_names[i]}: </label><br>
    <input type="text" id={stat_names[i]} name={stat_names[i]} class="input-text-box" value=""><br>
    ''' 
    if (i+1)%4==0: 
        output_html += '''\n</div> 
<br> 
<div class="input-row"> ''' 
output_html += '\n</div>'
 
print(output_html) 

<div class="input-row">
<label for="Age" class="input-label"> Age: </label><br>
    <input type="text" id=Age name=Age class="input-text-box" value=""><br>
    <label for="GP" class="input-label"> GP: </label><br>
    <input type="text" id=GP name=GP class="input-text-box" value=""><br>
    <label for="MPG" class="input-label"> MPG: </label><br>
    <input type="text" id=MPG name=MPG class="input-text-box" value=""><br>
    <label for="FG" class="input-label"> FG: </label><br>
    <input type="text" id=FG name=FG class="input-text-box" value=""><br>
    
</div> 
<br> 
<div class="input-row"> <label for="FGA" class="input-label"> FGA: </label><br>
    <input type="text" id=FGA name=FGA class="input-text-box" value=""><br>
    <label for="FG%" class="input-label"> FG%: </label><br>
    <input type="text" id=FG% name=FG% class="input-text-box" value=""><br>
    <label for="3P" class="input-label"> 3P: </label><br>
    <input type="text" id=3P name=3P class="input-text-box" value=""><br>
 